In [9]:
import pandas as pd
import csv 
import geopandas as gpd
from datetime import timedelta
from shapely.geometry import Point
from shapely import wkt
import glob
import matplotlib.pyplot as plt

In [10]:
#read csv files and adjust to datetime
C_im_date = pd.read_csv('../data/Cienega/CienegaImageryDates.csv', parse_dates=['date'])
C_sur_date = pd.read_csv('../data/Cienega/Cienega_survey_dates.csv', parse_dates=['Cienega date'])
C_sur_date['Cienega date'] = pd.to_datetime(C_sur_date['Cienega date'])
C_im_date['date'] = pd.to_datetime(C_im_date['date'])

C_hyd = pd.read_csv('../data/Cienega/CienegaHydroData.csv')
C_hyd['datetime'] = pd.to_datetime(C_hyd['datetime'])

C_precipitation = pd.read_csv('../data/Cienega/daymet_precip.csv')
C_precipitation['system:time_start'] = pd.to_datetime(C_precipitation['system:time_start'])
C_precipitation.rename( columns={'00000000000000000000':'P','system:time_start':'day'}, inplace=True )

C_sur_date = C_sur_date.dropna(subset=['Cienega date'])
C_im_date = C_im_date.dropna(subset=['date'])

C_surveyData = pd.read_csv('../data/Cienega/Cienega_surveyData.csv')
C_surveyData['Year'] = pd.to_datetime(C_surveyData['Year'])


In [11]:
#finding closest matching dates between survey and imagery
matching_dates = []
tolerance = timedelta(days = 5)


for date1 in C_sur_date['Cienega date']:
    exact_date = False
    tol = False 
    for date2 in C_im_date['date']:
        if date1 == date2:
            matching_dates.append({'Survey': date1, 'Imagery': date2})
            exact_date = True
    if not exact_date:
        for date2 in C_im_date['date']:
            if abs(date1 - date2) <= tolerance:
                matching_dates.append({'Survey': date1, 'Imagery': date2})
                tol = True
        if not tol:
            for date2 in C_im_date['date']:
                if abs(date1-date2) < timedelta(days = 10): 
                    matching_dates.append({'Survey': date1, 'Imagery': date2})


matching_dates_df = pd.DataFrame(matching_dates)



In [12]:
#merging dfs to use to determine imagery dates for survey dates
C_datessurData = pd.merge(matching_dates_df, C_hyd, left_on = 'Survey', right_on = 'datetime', how = 'left')
C_datesimData = pd.merge(matching_dates_df, C_hyd, left_on = 'Imagery', right_on = 'datetime')
C_datessurData = C_datessurData.drop(columns = ['Imagery','datetime'])
C_datesimData = C_datesimData.drop(columns = ['Survey','datetime'])

In [13]:
#sum precipitation for dates in between survey and imagery
def sum_pdatesbetween(d1, d2):
    r = pd.date_range(start=min(d1,d2), end=max(d1,d2))
    return C_hyd[C_hyd['datetime'].isin(r)]['P [mm]'].sum()

In [14]:
#making a dataframe to determine which imagery dates to use
Ch = pd.DataFrame([])

Ch['Survey'] = matching_dates_df['Survey']
Ch['Imagery'] = matching_dates_df['Imagery']
Ch['sum_P'] = [sum_pdatesbetween(C_datessurData.loc[i, 'Survey'], C_datesimData.loc[i, 'Imagery']) for i in range(len(Ch))]
Ch['Q_diff [%]'] = (C_datessurData['Q [mm/d]'] - C_datesimData['Q [mm/d]']) / C_datessurData['Q [mm/d]'] * 100
Ch['Use/not'] = ['use', 'use', 'use', 'use', 'not', 'not', 'use?',
                 'not', 'only option', 'not', 'not', 'use', 'not',
                 'not', 'not', 'use', 'not', 'not', 'not', 'not',
                 'use', 'use', 'use', 'only option','use', 'use', 
                 'use','use', 'not', 'not', 'not', 'not', 'use', 
                 'not', 'use', 'use', 'use', 'use', 'use', 'not', 
                 'not', 'not', 'not', 'use', 'not', 'not', 'not', 
                 'not', 'not', 'use', 'not', 'not', 'not', 'use', 
                 'not', 'use', 'use']


conditions = (Ch['sum_P'] > 3) | (Ch['Q_diff [%]'] > 8) | (Ch['Use/not'] == 'not')

Ch = Ch[~conditions]

Ch = Ch.drop(columns=['Use/not'])


In [15]:
Ch.to_csv('../data/Cienega/Cienega_survey_imagery_HydroData.csv', encoding='utf-8', index=False)

In [16]:
#reading surveydata, making it into a geodataframe and adding x and y from the geometry to facilitate merge later 
C_surveyData['geometry'] = C_surveyData['geometry'].apply(wkt.loads)
C_surveyData = C_surveyData.set_geometry('geometry')
gdf = gpd.GeoDataFrame(C_surveyData, geometry = 'geometry')#, crs='EPSG:26912')
gdf['x'] = gdf.geometry.x
gdf['y'] = gdf.geometry.y
gdf = gdf[['geometry', 'x', 'y', 'wetdry', 'Year']]
gdf['Year'] = pd.to_datetime(gdf['Year'])

gdf

,geometry,x,y,wetdry,Year
0,POINT (533719.252 3542427.373),533719.252411,3.542427e+06,dry,2004-06-25
1,POINT (533719.787 3542422.402),533719.786671,3.542422e+06,dry,2004-06-25
2,POINT (533720.321 3542417.431),533720.320931,3.542417e+06,dry,2004-06-25
3,POINT (533720.855 3542412.459),533720.855191,3.542412e+06,dry,2004-06-25
4,POINT (533721.389 3542407.488),533721.389451,3.542407e+06,dry,2004-06-25
...,...,...,...,...,...
237685,POINT (533717.227 3542446.217),533717.227336,3.542446e+06,dry,2013-09-05
237686,POINT (533717.762 3542441.246),533717.761591,3.542441e+06,dry,2013-09-05
237687,POINT (533718.296 3542436.274),533718.295846,3.542436e+06,dry,2013-09-05
237688,POINT (533718.830 3542431.303),533718.830102,3.542431e+06,dry,2013-09-05


In [17]:
# using new data for precipitation 
C_new_hyd = C_hyd.merge(C_precipitation, left_on = 'datetime', right_on = 'day')
C_new_hyd = C_new_hyd.drop(columns = ['day', 'P [mm]'])
C_new_hyd.rename( columns={'P':'P [mm]'}, inplace=True )
C_new_hyd.set_index(['datetime'], inplace = True)

In [18]:
# function to define assumptions around dates to choose, based on streamflow and precipitation
# Q_condition could be completely removed
def tolerance(Q_P_data, date, start, adjust, tolerance_p, P_condition = -999, Q_condition = -999):
    
    sub_grupp = Q_P_data.copy()
    
    if adjust == 'start':      
        sub_grupp = Q_P_data.loc[start:].copy()       
        
    elif adjust == 'end': #reverse index to loop backwards
        sub_grupp = sub_grupp.loc[:start].copy().iloc[::-1]        
    
    else:
        print('Invalid adjust parameter. Please use "start" or "end"')
        return

    
    # Reset index if reversed
    sub_grupp.reset_index(inplace=True)

    #creating a column for difference in streamflow
    sub_grupp['Q_diff'] = sub_grupp['Q [mm/d]'].diff().fillna(0)

    #checking to see if streamflow is overall decreasing, but a tolerance of x for any daily increase
    if adjust == 'start':
        sub_grupp['tolerance_condition'] = (sub_grupp.Q_diff < tolerance_p * sub_grupp['Q [mm/d]'])

    if adjust == 'end':
        sub_grupp['tolerance_condition'] = (sub_grupp.Q_diff > -tolerance_p * sub_grupp['Q [mm/d]'])

    
    if P_condition == -999 == Q_condition:
        print('not a valid condition')
        return 
        
    elif P_condition == -999:
        if Q_condition > 0:
            sub_grupp['condition'] = sub_grupp['Q [mm/d]'] > Q_condition
        else:
            sub_grupp['condition'] = sub_grupp['Q [mm/d]'] < -Q_condition
            
    elif Q_condition == -999:
        if P_condition > 0:
            sub_grupp['condition'] = sub_grupp['P [mm]'] > P_condition
        else:
            sub_grupp['condition'] = sub_grupp['P [mm]'] < -P_condition
            
    else:
        if (Q_condition > 0) & (P_condition > 0):
            sub_grupp['condition'] = (sub_grupp['Q [mm/d]'] > Q_condition) & (sub_grupp['P [mm]'] > P_condition)
        elif (Q_condition < 0) & (P_condition > 0):
            sub_grupp['condition'] = (sub_grupp['Q [mm/d]'] < -Q_condition) & (sub_grupp['P [mm]'] > P_condition)           
        elif (Q_condition > 0) & (P_condition < 0):
            sub_grupp['condition'] = (sub_grupp['Q [mm/d]'] > Q_condition) & (sub_grupp['P [mm]'] < -P_condition)            
        else:
            sub_grupp['condition'] = (sub_grupp['Q [mm/d]'] < -Q_condition) & (sub_grupp['P [mm]'] < -P_condition)

    # where both conditions are true
    yesgroup = sub_grupp[(sub_grupp['condition'] == True) & (sub_grupp['tolerance_condition'] == True)] 
       
    
    if len(yesgroup) == 0:
        print('No data where conditions are met')
        return pd.DataFrame()

    #the first instance where conditions are false after conditions hev been met 
    nogroup = sub_grupp[(sub_grupp['condition'] == False) | (sub_grupp['tolerance_condition'] == False)]  
    
    if len(nogroup) == 0:
        print('nogroup = 0')
        return sub_grupp.loc[yesgroup.index[0]:]

    if yesgroup.index[0] < nogroup.index[0]:
        print('everything is fine')
        return sub_grupp.loc[:nogroup.index[0]]
    
    else:
        print('No valid range found between yesgroup and nogroup indices')
        return pd.DataFrame()



In [19]:
#gdf['geometry_wkt'] = gdf['geometry'].apply(lambda geom: geom.wkt)
# assuming perennial reaches
perennial = pd.DataFrame(gdf.groupby('geometry')['wetdry'].apply(lambda x: sum(x == 'wet'))).reset_index(drop=False)
perennialcount = pd.DataFrame(gdf.groupby('geometry')['wetdry'].count()).reset_index(drop=False)

# whichever number is reasonable based on data?
perennial = perennial[(perennial['wetdry'] >= (perennialcount['wetdry']))]

#assume always wet
perennial = perennial.assign(wetdry = 'wet')
#perennial['geometry'] = perennial['geometry_wkt'].apply(wkt.loads)
#perennial = perennial.drop(columns=['geometry_wkt'])

#perennial['geometry'] = perennial['geometry'].apply(wkt.loads)
gdf_perennial = gpd.GeoDataFrame(perennial, geometry = 'geometry')#, crs='EPSG:26912')
gdf_perennial['x'] = gdf_perennial.geometry.x
gdf_perennial['y'] = gdf_perennial.geometry.y

#making the gdf matching the perennial reaches to all the imagery dates available 
imagery_perennial = pd.concat([gdf_perennial.assign(imagery = date) for date in C_im_date['date']], ignore_index=True)
imagery_perennial = imagery_perennial[~imagery_perennial['imagery'].isin(Ch['Imagery'])]
imagery_perennial['assumption'] = len(imagery_perennial)*['assumed perennial']
imagery_perennial 

,geometry,wetdry,x,y,imagery,assumption


In [20]:
#assuming wet stretches for the dates before
#assumption is made with 5 % difference in streamflow and for dates before survey when in a recession 

wet_list = []

for date in C_surveyData['Year'].unique():
            
    wet1 = tolerance(C_new_hyd, 'datetime', date, 'end', 0.05, Q_condition = -999, P_condition = -1)
    if len(wet1) == 0:
        print('wet1 is empty')
        continue
    wet1 = wet1[~wet1['datetime'].isin(Ch['Imagery'])]
    wet_imagery = pd.merge(wet1, C_im_date, left_on = ['datetime'], right_on = ['date'], how = 'inner')
    #print(len(wet_imagery))
    wet_points = pd.DataFrame(gdf[gdf['Year']== (date)].groupby('geometry')['wetdry'].apply(lambda x: sum(x == 'wet'))).reset_index(drop = False)
    wet_points = wet_points[(wet_points['wetdry'] == 1)]
    wet_points = wet_points.assign(wetdry = 'wet')
    wet_im_points = [wet_points.assign(imagery = date) for date in wet_imagery['date']]
        
    try:
        wet = pd.concat(wet_im_points).drop(columns = ['level_1'])
        wet_list.append(wet)
    except:
        if len(wet_im_points)==0:
            print('No data for date '+ date.strftime('%Y-%m-%d'))
        else:
            wet = wet_im_points[0]
            wet_list.append(wet)
        

wet_df = pd.concat(wet_list)


wet_df['assumption'] = len(wet_df)*['assumed wet']




No data where conditions are met
wet1 is empty
everything is fine
No data where conditions are met
wet1 is empty
everything is fine
No valid range found between yesgroup and nogroup indices
wet1 is empty
No data where conditions are met
wet1 is empty
No data where conditions are met
wet1 is empty
No data where conditions are met
wet1 is empty
No data where conditions are met
wet1 is empty
No data where conditions are met
wet1 is empty
everything is fine
No data where conditions are met
wet1 is empty
everything is fine
No data for date 2017-09-19
everything is fine
No data for date 2016-06-03
No data where conditions are met
wet1 is empty
No data where conditions are met
wet1 is empty
everything is fine
No data for date 2018-03-23
No data where conditions are met
wet1 is empty
No data where conditions are met
wet1 is empty
No data where conditions are met
wet1 is empty
everything is fine
No data where conditions are met
wet1 is empty
everything is fine
everything is fine
everything is f

In [21]:
#assuming dry stretches for the dates after
#assumption is made with 5 % difference in streamflow and for dates after survey when in a recession 

dry_list = []


for date in C_surveyData['Year'].unique():
            
    dry1 = tolerance(C_new_hyd, 'datetime', date, 'start', 0.05, Q_condition = -999, P_condition = -1)
    if len(dry1) == 0:
        print('wet1 is empty')
        continue
    dry1 = dry1[~dry1['datetime'].isin(Ch['Imagery'])]
    dry_imagery = pd.merge(dry1, C_im_date, left_on = ['datetime'], right_on = ['date'], how = 'inner')
        #print(len(wet_imagery))
    dry_points = pd.DataFrame(gdf[gdf['Year']== (date)].groupby('geometry')['wetdry'].apply(lambda x: sum(x == 'dry'))).reset_index(drop = False)
    dry_points = dry_points[(dry_points['wetdry'] == 1)].assign(wetdry = 'dry')
    dry_im_points = [dry_points.assign(imagery = date) for date in dry_imagery['date']]
        
    try:
        dry = pd.concat(dry_im_points).drop(columns = ['level_1'])
        dry_list.append(dry)
        
    except:
        if len(dry_im_points)==0:
            print('No data for date '+ date.strftime('%Y-%m-%d'))
        else:
            dry = dry_im_points[0]
            dry_list.append(dry)
        #print(len(dry))
        

dry_df = pd.concat(dry_list)

dry_df['assumption'] = len(dry_df)*['assumed dry']





everything is fine
No data for date 2004-06-25
everything is fine
No data for date 2022-09-08
everything is fine
No data for date 2012-12-12
everything is fine
No valid range found between yesgroup and nogroup indices
wet1 is empty
everything is fine
No data for date 2008-09-16
everything is fine
No data for date 2010-06-17
everything is fine
No data for date 2014-06-13
everything is fine
No data for date 2005-06-23
everything is fine
No data for date 2009-06-11
everything is fine
everything is fine
No data for date 2011-03-31
everything is fine
everything is fine
No data for date 2016-06-03
everything is fine
No data for date 2004-03-03
everything is fine
No data for date 2001-06-26
everything is fine
No data for date 2018-03-23
everything is fine
No data for date 2010-03-25
everything is fine
No data for date 2011-12-23
everything is fine
No data for date 2012-03-13
everything is fine
everything is fine
No data for date 2010-12-14
everything is fine
everything is fine
everything is f

In [23]:
# concatenate all dfs with assumptions and survey matched to imagery dates and turn to gdf
gdf['assumption'] = len(gdf)*['survey/imagery match']
gdf_imagery = pd.merge(gdf, Ch, left_on = 'Year', right_on = 'Survey', how = 'left')
gdf_imagery = gdf_imagery.drop(columns=['Survey', 'sum_P', 'Q_diff [%]', 'Year'])
all_expanded = pd.concat([gdf_imagery, imagery_perennial, wet_df, dry_df])
all_expanded = gpd.GeoDataFrame(all_expanded, geometry = 'geometry')#, crs='EPSG:26912')
all_expanded['x'] = all_expanded.geometry.x
all_expanded['y'] = all_expanded.geometry.y
all_expanded = all_expanded.rename(columns = {'imagery':'date_first'})
all_expanded['date'] = all_expanded['Imagery'].combine_first(all_expanded['date_first'])
all_expanded

,geometry,x,y,wetdry,assumption,Imagery,date_first,date
0,POINT (533719.252 3542427.373),533719.252411,3.542427e+06,dry,survey/imagery match,NaT,NaT,NaT
1,POINT (533719.787 3542422.402),533719.786671,3.542422e+06,dry,survey/imagery match,NaT,NaT,NaT
2,POINT (533720.321 3542417.431),533720.320931,3.542417e+06,dry,survey/imagery match,NaT,NaT,NaT
3,POINT (533720.855 3542412.459),533720.855191,3.542412e+06,dry,survey/imagery match,NaT,NaT,NaT
4,POINT (533721.389 3542407.488),533721.389451,3.542407e+06,dry,survey/imagery match,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...
2636,POINT (539511.378 3540289.598),539511.377590,3.540290e+06,dry,assumed dry,NaT,2018-09-09,2018-09-09
2637,POINT (539516.317 3540290.375),539516.316935,3.540290e+06,dry,assumed dry,NaT,2018-09-09,2018-09-09
2638,POINT (539396.748 3540256.611),539396.747956,3.540257e+06,dry,assumed dry,NaT,2018-09-09,2018-09-09
2639,POINT (539392.098 3540254.772),539392.098410,3.540255e+06,dry,assumed dry,NaT,2018-09-09,2018-09-09


In [27]:
#reading and concatenating the processed imagery 
path = '../data/Cienega/processed_imagery'

processed_imagery = glob.glob(path + '/*.csv')
processed_imagery.sort(key = lambda x: int(x.split('_buffer_')[1].split('.')[0]))
con_ready_imagery = []
for processed in processed_imagery:
    df= pd.read_csv(processed)
    con_ready_imagery.append(df)

concatenated = pd.concat(con_ready_imagery)

In [28]:
concatenated['geometry'] = concatenated['geometry'].apply(wkt.loads)
gdf_processed = gpd.GeoDataFrame(concatenated, geometry = 'geometry')#, crs='EPSG:26912')
gdf_processed['date'] = pd.to_datetime(gdf_processed['date'], format='%Y%m%d')
gdf_processed['x'] = gdf_processed.geometry.x
gdf_processed['y'] = gdf_processed.geometry.y
gdf_processed = gdf_processed.drop_duplicates()

In [29]:
precision = 6
all_expanded['x'] = all_expanded['x'].round(precision)
all_expanded['y'] = all_expanded['y'].round(precision)
gdf_processed['x'] = gdf_processed['x'].round(precision)
gdf_processed['y'] = gdf_processed['y'].round(precision)

In [30]:
merged = all_expanded.merge(gdf_processed, on=['date', 'x', 'y'])

In [31]:
merged = merged.drop(columns = ['geometry_x', 'geometry_y', 'Imagery', 'date_first']) 
merged_sorted = merged.sort_values(by='date')
merged_sorted = merged_sorted.drop_duplicates()
merged_sorted.dropna(inplace= True)
merged_sorted

,x,y,wetdry,assumption,date,blue,green,red,NIR,missing,NDWI,p
67535,534451.929516,3.542234e+06,dry,survey/imagery match,2016-09-22,1124.56,1395.33,1837.78,2920.00,0,-0.35,0
67533,534446.929543,3.542234e+06,dry,survey/imagery match,2016-09-22,1089.56,1368.11,1801.33,2846.22,0,-0.35,0
67531,534441.929569,3.542234e+06,dry,survey/imagery match,2016-09-22,1073.56,1343.22,1780.00,2792.22,0,-0.35,0
67537,534455.700434,3.542231e+06,dry,survey/imagery match,2016-09-22,944.22,1228.11,1619.22,2888.56,0,-0.40,0
67529,534436.929596,3.542234e+06,dry,survey/imagery match,2016-09-22,997.56,1270.00,1691.44,2742.11,0,-0.37,0
...,...,...,...,...,...,...,...,...,...,...,...,...
222793,532264.228957,3.543183e+06,dry,assumed dry,2023-06-11,1069.11,1500.78,1842.22,2806.89,0,-0.30,0
222791,532265.567543,3.543178e+06,dry,assumed dry,2023-06-11,1065.11,1512.44,1836.00,2769.11,0,-0.29,0
222789,532266.906128,3.543173e+06,dry,assumed dry,2023-06-11,1076.78,1495.44,1825.89,2851.56,0,-0.31,0
222787,532262.890372,3.543187e+06,dry,assumed dry,2023-06-11,1094.89,1502.78,1848.11,2786.89,0,-0.30,0


In [33]:
start = 0
splitnum = 10
for i in range(1,splitnum+1):
    newstart = int(len(merged_sorted)/splitnum*i)
    merged_sorted.iloc[start:newstart].to_csv('../data/Cienega/processed_assumptions/processed_with_dates_and_assumptions'+str(i)+'.csv',index=False,
                      float_format='%.2f')
    start = newstart
